# Импорт библиотек + функции

In [71]:
import requests
import json
import time
import os
import json
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.options.mode.chained_assignment = None
from ipywidgets import IntProgress
from IPython.display import display
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
CLEANR = re.compile('<.*?>') 

 
def getPage(param, page = 0):
    
    param['page'] = page
    params = param
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data

In [72]:
def transform(df, vacancy_details):
    df['low_salary'] = 0
    df['high_salary'] = 0
    df['requirement'] = '-'
    df['responsibility'] = '-'
    lst_names = ['area', 'employer', 'schedule']
    for i in range(len(df)):
        for item in vacancy_details:
            if item in lst_names:
                df[item][i] = df[item][i]['name']
            elif item == 'salary' and df[item][i] != None:
                if df[item][i]['from'] != None and df[item][i]['to'] !=None:
                    df['low_salary'][i] = df[item][i]['from']
                    df['high_salary'][i] = df[item][i]['to']
                if df[item][i]['to'] == None and df[item][i]['from'] != None:
                    df['low_salary'][i] = df[item][i]['from']
                    df['high_salary'][i] = df[item][i]['from']
                elif df[item][i]['from'] == None and df[item][i]['to'] != None :
                    df['low_salary'][i] = df[item][i]['to']
                    df['high_salary'][i] = df[item][i]['to']
            elif item == 'snippet':
                df['requirement'][i] = df[item][i]['requirement']
                df['responsibility'][i] = df[item][i]['responsibility']
    df = df.drop(columns=['salary','snippet']) 
                    
    return df

In [73]:
def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '', raw_html)
  return cleantext

# Ключевые слова

In [74]:
key_words = {'Конкурентоспособная оплата труда и выше': ['конкурент', 'оплата'],
 'Достойная оплата труда': ['достойн', 'оплата'],
 'Справедливая оплата труда ': ['справедл', 'оплата'],
 'Рост зп по итогам оценки': ['рост', 'заработной', 'оплаты'],
 'Возможность дополнительного заработка': ['доп', 'заработ'],
 'Квартальные премии': ['квартал', 'преми'],
 'Ежегодные премии': ['ежегод', 'преми'],
 'Премии за стаж': ['премия', '', 'стаж'],
 'Премии по итогам работы за месяц': ['премия', '', 'итог', 'работы'],
 'Приведи друга': ['приведи', 'друга'],
 'Премии по результатам работы': ['премия', 'результат'],
 'Премии за участие в проектах': ['преми', 'проект'],
 'Бонус "приведи друга"': ['приведи', 'друга'],
 'Надбавки за стаж': ['надбавк', 'стаж'],
 'Программа "Приведи друга"': ['приведи', 'друга'],
 'График работы сменный': ['сменный', 'график'],
 'График работы 5/2 ': ['5/2'],
 'График работы 2/2 сменный': ['2/2'],
 'График работы сменный(3/2 день, 2/2 ночь)': ['сменный', 'график'],
 'Гибкое начало дня': ['гибкое', 'начало'],
 'Удаленность/доставка': ['доставка'],
 'Душевые, санитарные комнаты, раздевалки и комнаты приема пищи': ['душев'],
 'Предоставление общежития в регионах': ['общежит'],
 'Оборудование для работы от работодателя': ['оборудование', 'работ'],
 'Планшет для работы': ['планшет'],
 'Удаленка/микс какой формат работы': ['удаленный', 'формат'],
 'Крутой офис… уточняем': ['крутой', 'офис'],
 'График работы гибкий': ['гибкий', 'график'],
 'Релокационный пакет': ['релокац', 'пакет'],
 'Современное оборудование и ПО на выбор;': ['соврменное', 'оборудование'],
 'Есть возможность удаленной работы': ['возможность', 'удаленной', 'работы'],
 'Возможность частично удаленной работы': ['возможность',
  'частично',
  'удаленной',
  'работы'],
 'Комфортный офис': ['комфортный', 'офис'],
 'Скидки на продукцию и у партнеров': ['скидки'],
 'Страхование жизни': ['страхование', 'жизни'],
 'Льготные условия приобретения полисов ДМС': ['льготные', 'условия', 'ДМС'],
 'ДМС (указываем отдельно варианты, расширенный, софинансирование, франшиза, все отдельными строками)': ['ДМС'],
 'ОМС': ['ОМС'],
 'Льготные условия кредитования': ['льготные', 'условия', 'кредит'],
 'Льготные условия автострахования': ['Льготные',
  'условия',
  'автострахования'],
 'Корпоративные пенсионные программы': ['Корпоративные',
  'пенсионные',
  'программы'],
 'Кофе, фрукты, печенье и прочее за счет работодателя': ['вкусняшки'],
 'Помощь в трудных жизненных ситуациях': ['помощь', 'трудных', 'ситуациях'],
 'Стархование жизни ближайших родственников': ['страхование', 'родственников'],
 'Страхование от несчастного случая, инвалидности': ['страхование',
  '',
  'несчастного',
  'инвалидности'],
 'Страхование для выезжающих за рубеж': ['страхование', 'выезжающих', 'рубеж'],
 'ДМС включая стоматологию ': ['ДМС', 'стоматолог'],
 'Доплата до среднего лист нетрудоспособности': ['доплата',
  '',
  'среднего',
  'нетрудоспособности'],
 'Льготное кредитование ипотека': ['льгот', 'ипотека'],
 'Льготное кредитование потребительский кредит': ['льгот', 'кредит'],
 'Компенсация путевок в лагеря': ['компенсация', 'путевок'],
 'ДМС ': ['ДМС'],
 'Льготные условия по ДМС для семьи': ['ДМС', 'семь'],
 'Льготное кредитование и страхование': ['льгот', 'кредит', 'страхование'],
 'Льготы на КАСКО': ['КАСКО'],
 'Специальные условия по вкладам': ['условия', 'по', 'вкладам'],
 'Рождение ребенка': ['Рождение', 'ребенка'],
 'Компенсация проезда к месту отдыха 1 раз в 2 года': ['компенсация',
  'проезда'],
 'Мед страхование для родственников': ['страхование', '', 'родственников'],
 'Спец. одежда': ['спец', 'одежда'],
 'Питание бесплатное': ['бесплатное', 'питание'],
 'Компенсация питания': ['Компенсация', 'питания'],
 'Столовая на территории': ['столовая'],
 'Компенсация сотовой связи': ['компенсация', 'связи'],
 'Компенсация транспортных расходов': ['компенсация', 'транспорт'],
 'Компенсация аренды авто': ['компенсация', 'аренд', 'авто'],
 'Корпоративный автомобиль': ['Корпоративный', 'автомобиль'],
 'Возможности обучения': ['обучение'],
 'Карьерный рост ': ['карьерный', 'рост'],
 'Компенсация обучения анг языка': ['обучение', 'английск'],
 'Сложные задачи': ['сложные', 'задачи'],
 'Обучение и сертификация за счет компании': ['обучение', 'за', 'счет'],
 'Участие в конференциях, вебинарах, сообществах': ['участие', 'в', 'конф'],
 'Крутые коллеги, готовые обучать и делиться': ['коллеги'],
 'Крутые наставники': ['наставники'],
 'ИПР (Индивидуальный план развития)': ['план', 'развития'],
 'Ежегодная оценка и ИПР': ['ежегодная', 'оценка'],
 'Обучение анг языку в офисе': ['обучение', 'язык'],
 'Возможность международной карьеры': ['международ', 'карьер'],
 'Программы обучения иностранному языку': ['программы', 'обучения', 'язык'],
 'Новогодние подарки детям сотрудников, подарки по случаю рождения детей, корпоративные праздники': ['новогодние',
  'подарки'],
 'Корпоративные праздники, выезды на природу': ['корпоративные', 'праздники'],
 'Подарки детям': ['подарки', 'детям'],
 'Корпоративные конкурсы и мероприятия': ['корпоративные', 'конкурсы'],
 'Корпоративные праздники, подарки детям': ['корпоративные', 'праздники'],
 'Работаем в гибких командах, используем продуктовую можель разработки': ['гибкие',
  'команд'],
 'Agile, scrum': ['Agile'],
 'Скидки на абонементы в спортзалы': ['скидки', 'спортзал'],
 'Компенсация спорта': ['Компенсация', 'спор'],
 'Возможность стать акционером компании': ['акционер', 'компани'],
 'Собственный спортзал': ['Собственный', 'спортз'],
 'Йога в офисе': ['йога'],
 'Психолог': ['психолог'],
 'Обучение финансовой грамотности': ['финанс', 'грамотн'],
 'возможность заниматься корпоративным спортом (футбол, волейбол)': ['корпорат',
  'спорт'],
 'Бесплатная электронная библиотека': ['библиотека'],
 'Скидки в тур-агенствах и других организациях': ['скидки',
  'тур', 'агенства'],
 'Стоматолог': ['стоматолог'],
 'Терапевт': ['терапевт'],
 'Автошкола на территории компании': ['автошкола']}

# Работа с параметрами + ввод профилей должности

In [89]:
parametr =   [{
        'text': 'Мерчендайзер', # Наименование вакансии
        'area': 66, # Город поиска
        'page': 0, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }]

job_profile = {'Логист': 'Ведение операционного дня.Контроль отгрузок и ТС по маршруту.Маршрутизация.Ведение отчетности.Взаимодействие со смежными отделами компании',
            'Продавец-консультант': 'Продажи. Консультирование клиентов. Контроль дебиторской задолженности'}

# Работаем с API HeadHunter

In [90]:
ind = 0
vacancy_details = ['name', 'area', 'salary', 'alternate_url', 'employer', 'snippet', 'schedule']
df = pd.DataFrame(columns= list(vacancy_details))
df['vacancy_id'] = '-'
for param in parametr:
    for page in range(0, 1): #здесь можно менять кол-во страниц для поиска
        data = []
        e = json.loads(getPage(param, page))
        data.append(e)
        
        for i in range(len(data)):
            for j in range(len(data[i]['items'])):
                df.loc[ind] = data[i]['items'][j]
                df['vacancy_id'][ind] = param['text']
                ind+=1
                
    
    

In [49]:
df = transform(df, vacancy_details)
df['description'] = '-'

In [50]:
max_count = len(df['alternate_url'])
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
for i in range(len(df)):
    f.value += 1
    URL = (df['alternate_url'][i])
    req = Request(URL, headers = headers)
    try:
        oururl = urlopen(req).read()
    except ConnectionResetError as e:
        print('Ошибка соединения')
        df['description'][i] = 'Ошибка соединения'
        continue
    soup = BeautifulSoup(oururl)
    data = json.loads(soup.find('script', type='application/ld+json').text)
    description = cleanhtml(data['description'])
    df['description'][i] = description
    time.sleep(5)
    

IntProgress(value=0, max=200)

# Разносим единицы по пунктам

In [52]:
for key in key_words.keys():
    df[key] = ''
    for j in range(len(df['description'])):
        count = 0
        for i in range(len(key_words[key])):
            if key_words[key][i] in df['description'][j]:
                count+=1
        if count == len(key_words[key]):
            df[key][j] = 1
        


C:\Users\cyril\AppData\Local\Temp/ipykernel_1296/2911190588.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key] = ''
C:\Users\cyril\AppData\Local\Temp/ipykernel_1296/2911190588.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[key] = ''
C:\Users\cyril\AppData\Local\Temp/ipykernel_1296/2911190588.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. T

# Работаем над профилем должности

In [64]:
df['Index'] = 0
for i in range(len(df['vacancy_id'])):
    for profile in job_profile:
        if profile == df['vacancy_id'][i]:
            df['Index'][i] = fuzz.WRatio(df['responsibility'][i], job_profile[profile])
            
        

# Сохраняем файл в excel

In [67]:
writer = pd.ExcelWriter('parser_HH.xlsx')
df.to_excel(writer)
writer.save()

In [91]:
df

,name,area,salary,alternate_url,employer,snippet,schedule,vacancy_id
0,Мерчендайзер,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 45000, 'to': None, 'currency': 'RUR',...",https://hh.ru/vacancy/71438993,"{'id': '83056', 'name': 'Action', 'url': 'http...","{'requirement': 'Опыт работы любой, проводим о...","{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
1,Мерчендайзер магазинов,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 45000, 'to': 45000, 'currency': 'RUR'...",https://hh.ru/vacancy/71417951,"{'id': '3272879', 'name': 'Маруся Фуд', 'url':...",{'requirement': 'Опыт работы в сфере розничных...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
2,Мерчендайзер,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': None, 'to': 60000, 'currency': 'RUR',...",https://hh.ru/vacancy/70895798,"{'id': '151459', 'name': 'Компания БФК', 'url'...",{'requirement': 'Приветствуется наличие профил...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
3,Мерчендайзер,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 30300, 'to': 30300, 'currency': 'RUR'...",https://hh.ru/vacancy/71199347,"{'id': '56478', 'name': 'Сладкая жизнь, компан...",{'requirement': 'Рассматриваются кандидаты и б...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
4,Супервайзер мерчендайзеров,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 67000, 'to': None, 'currency': 'RUR',...",https://hh.ru/vacancy/71697366,"{'id': '83056', 'name': 'Action', 'url': 'http...",{'requirement': 'Грамотная речь. Пользователь ...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
...,...,...,...,...,...,...,...,...
95,Продавец-Кассир в г. Нижний Новгород,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 33000, 'to': 36000, 'currency': 'RUR'...",https://hh.ru/vacancy/68733999,"{'id': '196621', 'name': 'FIX PRICE', 'url': '...",{'requirement': 'Наши преимущества: - Мы рады ...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
96,Продавец - Кассир в г. Нижний Новгород,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 33000, 'to': 36000, 'currency': 'RUR'...",https://hh.ru/vacancy/71756956,"{'id': '196621', 'name': 'FIX PRICE', 'url': '...",{'requirement': 'Наши преимущества: - Мы рады ...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
97,Продавец-консультант,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 40000, 'to': None, 'currency': 'RUR',...",https://hh.ru/vacancy/71104118,"{'id': '4934', 'name': 'билайн', 'url': 'https...",{'requirement': 'Что ты будешь делать: Стремле...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
98,Продавец-консультант,"{'id': '66', 'name': 'Нижний Новгород', 'url':...","{'from': 40000, 'to': None, 'currency': 'RUR',...",https://hh.ru/vacancy/70747587,"{'id': '4934', 'name': 'билайн', 'url': 'https...",{'requirement': 'Что ты будешь делать: Стремле...,"{'id': 'fullDay', 'name': 'Полный день'}",Мерчендайзер
